In [2]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
import csv
import base64
#import matplotlib.pyplot as plt
import io
import os
from math import floor

## web crawling packages
import time
from datetime import date
from datetime import datetime
import datetime
import requests
from lxml import html

In [9]:
## loading historical games
historical = pd.read_csv('nfl_historical_clean.csv')
historical = historical.drop(columns='Unnamed: 0')
historical = historical.drop_duplicates()
historical['year'] = historical['url'].str[-17:-13]
historical['year'] = historical['year'].astype(int)
today = date.today()
year = today.year
historical = historical[historical['year'] == year]

In [19]:
team = 'CLE'

new_DataFrame = historical[(historical['away_team'] == team) | (historical['home_team'] == team)]


stat_df_away = new_DataFrame[new_DataFrame['away_team'] == team]
stat_df_away = stat_df_away[['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'yards_per_pass_away', 'int_thrown_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'turnovers_away', 'fumbles_lost_away', 'defensive_td_away', 'possession_away', 'sack_yards_lost_away_occur', 'sack_yards_lost_away_yards', 'penalty_away_occur', 'penalty_away_yards']]
stat_df_away = stat_df_away.rename(columns = {'first_downs_away': 'first_downs', 'first_downs_passing_away': 'first_downs_passing', 'first_down_rushing_away': 'first_down_rushing', 'first_down_by_penalty_away': 'first_down_by_penalty', 'total_plays_away': 'total_plays', 'total_yards_away': 'total_yards', 'total_drives_away': 'total_drives', 'yards_per_play_away': 'yards_per_play', 'passing_away': 'passing', 'yards_per_pass_away': 'yards_per_pass', 'int_thrown_away': 'int_thrown', 'rushing_away': 'rushing', 'rushing_att_away': 'rushing_att', 'yards_per_rush_away': 'yards_per_rush', 'turnovers_away': 'turnovers', 'fumbles_lost_away': 'fumbles_lost', 'defensive_td_away': 'defensive_td', 'possession_away': 'possession', 'sack_yards_lost_away_occur': 'sack_yards_lost_occur', 'sack_yards_lost_away_yards': 'sack_yards_lost_yards', 'penalty_away_occur': 'penalty_occur', 'penalty_away_yards': 'penalty_yards'})


stat_df_home = new_DataFrame[new_DataFrame['home_team'] == team]
stat_df_home = stat_df_home[['first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'yards_per_pass_home', 'int_thrown_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'turnovers_home', 'fumbles_lost_home', 'defensive_td_home', 'possession_home', 'sack_yards_lost_home_occur', 'sack_yards_lost_home_yards', 'penalty_home_occur', 'penalty_home_yards']]
stat_df_home = stat_df_home.rename(columns = {'first_downs_home': 'first_downs', 'first_downs_passing_home': 'first_downs_passing', 'first_down_rushing_home': 'first_down_rushing', 'first_down_by_penalty_home': 'first_down_by_penalty', 'total_plays_home': 'total_plays', 'total_yards_home': 'total_yards', 'total_drives_home': 'total_drives', 'yards_per_play_home': 'yards_per_play', 'passing_home': 'passing', 'yards_per_pass_home': 'yards_per_pass', 'int_thrown_home': 'int_thrown', 'rushing_home': 'rushing', 'rushing_att_home': 'rushing_att', 'yards_per_rush_home': 'yards_per_rush', 'turnovers_home': 'turnovers', 'fumbles_lost_home': 'fumbles_lost', 'defensive_td_home': 'defensive_td', 'possession_home': 'possession', 'sack_yards_lost_home_occur': 'sack_yards_lost_occur', 'sack_yards_lost_home_yards': 'sack_yards_lost_yards', 'penalty_home_occur': 'penalty_occur', 'penalty_home_yards': 'penalty_yards'})

team_stats_all = stat_df_away.append(stat_df_home)
team_stats_agg = pd.DataFrame(team_stats_all.mean()).T
team_stats_agg['team'] = team
team_stats_agg


,first_downs,first_downs_passing,first_down_rushing,first_down_by_penalty,total_plays,total_yards,total_drives,yards_per_play,passing,yards_per_pass,...,rushing_att,yards_per_rush,turnovers,fumbles_lost,defensive_td,possession,sack_yards_lost_occur,sack_yards_lost_yards,penalty_occur,penalty_yards
0,20.727273,10.636364,8.545455,1.545455,62.090909,362.454545,10.090909,5.909091,205.636364,6.618182,...,29.909091,5.209091,1.0,0.454545,0.090909,31.591818,2.818182,17.272727,7.272727,66.0


In [22]:
team_list = historical['away_team'].unique()
len(team_list)

32

In [23]:
stat_agg_df = pd.DataFrame()

for i in team_list:
    team = i

    new_DataFrame = historical[(historical['away_team'] == team) | (historical['home_team'] == team)]

    stat_df_away = new_DataFrame[new_DataFrame['away_team'] == team]
    stat_df_away = stat_df_away[['first_downs_away', 'first_downs_passing_away', 'first_down_rushing_away', 'first_down_by_penalty_away', 'total_plays_away', 'total_yards_away', 'total_drives_away', 'yards_per_play_away', 'passing_away', 'yards_per_pass_away', 'int_thrown_away', 'rushing_away', 'rushing_att_away', 'yards_per_rush_away', 'turnovers_away', 'fumbles_lost_away', 'defensive_td_away', 'possession_away', 'sack_yards_lost_away_occur', 'sack_yards_lost_away_yards', 'penalty_away_occur', 'penalty_away_yards']]
    stat_df_away = stat_df_away.rename(columns = {'first_downs_away': 'first_downs', 'first_downs_passing_away': 'first_downs_passing', 'first_down_rushing_away': 'first_down_rushing', 'first_down_by_penalty_away': 'first_down_by_penalty', 'total_plays_away': 'total_plays', 'total_yards_away': 'total_yards', 'total_drives_away': 'total_drives', 'yards_per_play_away': 'yards_per_play', 'passing_away': 'passing', 'yards_per_pass_away': 'yards_per_pass', 'int_thrown_away': 'int_thrown', 'rushing_away': 'rushing', 'rushing_att_away': 'rushing_att', 'yards_per_rush_away': 'yards_per_rush', 'turnovers_away': 'turnovers', 'fumbles_lost_away': 'fumbles_lost', 'defensive_td_away': 'defensive_td', 'possession_away': 'possession', 'sack_yards_lost_away_occur': 'sack_yards_lost_occur', 'sack_yards_lost_away_yards': 'sack_yards_lost_yards', 'penalty_away_occur': 'penalty_occur', 'penalty_away_yards': 'penalty_yards'})
    
    stat_df_home = new_DataFrame[new_DataFrame['home_team'] == team]
    stat_df_home = stat_df_home[['first_downs_home', 'first_downs_passing_home', 'first_down_rushing_home', 'first_down_by_penalty_home', 'total_plays_home', 'total_yards_home', 'total_drives_home', 'yards_per_play_home', 'passing_home', 'yards_per_pass_home', 'int_thrown_home', 'rushing_home', 'rushing_att_home', 'yards_per_rush_home', 'turnovers_home', 'fumbles_lost_home', 'defensive_td_home', 'possession_home', 'sack_yards_lost_home_occur', 'sack_yards_lost_home_yards', 'penalty_home_occur', 'penalty_home_yards']]
    stat_df_home = stat_df_home.rename(columns = {'first_downs_home': 'first_downs', 'first_downs_passing_home': 'first_downs_passing', 'first_down_rushing_home': 'first_down_rushing', 'first_down_by_penalty_home': 'first_down_by_penalty', 'total_plays_home': 'total_plays', 'total_yards_home': 'total_yards', 'total_drives_home': 'total_drives', 'yards_per_play_home': 'yards_per_play', 'passing_home': 'passing', 'yards_per_pass_home': 'yards_per_pass', 'int_thrown_home': 'int_thrown', 'rushing_home': 'rushing', 'rushing_att_home': 'rushing_att', 'yards_per_rush_home': 'yards_per_rush', 'turnovers_home': 'turnovers', 'fumbles_lost_home': 'fumbles_lost', 'defensive_td_home': 'defensive_td', 'possession_home': 'possession', 'sack_yards_lost_home_occur': 'sack_yards_lost_occur', 'sack_yards_lost_home_yards': 'sack_yards_lost_yards', 'penalty_home_occur': 'penalty_occur', 'penalty_home_yards': 'penalty_yards'})

    team_stats_all = stat_df_away.append(stat_df_home)
    team_stats_agg_temp = pd.DataFrame(team_stats_all.mean()).T
    team_stats_agg_temp['team'] = team
    stat_agg_df = stat_agg_df.append(team_stats_agg_temp)
    print(i, "Done")

print("All Done")

DAL Done
PHI Done
PIT Done
MIN Done
SF Done
ARI Done
SEA Done
LAC Done
NYJ Done
JAX Done
CLE Done
MIA Done
GB Done
DEN Done
CHI Done
BAL Done
NYG Done
CIN Done
HOU Done
LAR Done
BUF Done
NE Done
LV Done
NO Done
ATL Done
TEN Done
KC Done
DET Done
CAR Done
WSH Done
IND Done
TB Done
All Done


In [24]:
stat_agg_df.head()

,first_downs,first_downs_passing,first_down_rushing,first_down_by_penalty,total_plays,total_yards,total_drives,yards_per_play,passing,yards_per_pass,...,yards_per_rush,turnovers,fumbles_lost,defensive_td,possession,sack_yards_lost_occur,sack_yards_lost_yards,penalty_occur,penalty_yards,team
0,23.727273,15.090909,6.727273,1.909091,68.454545,419.272727,11.0,6.136364,287.272727,7.536364,...,4.490909,1.454545,0.727273,0.363636,31.832727,1.727273,10.636364,7.727273,66.363636,DAL
0,20.750000,9.583333,10.000000,1.166667,62.083333,355.583333,10.5,5.716667,197.166667,6.516667,...,5.033333,0.833333,0.333333,0.416667,29.123333,1.750000,10.750000,6.916667,57.166667,PHI
0,19.700000,12.300000,5.200000,2.200000,65.900000,328.000000,11.4,5.010000,237.300000,5.910000,...,3.620000,1.100000,0.600000,0.100000,31.269000,2.100000,12.800000,6.600000,47.100000,PIT
0,21.300000,12.700000,6.600000,2.000000,67.000000,387.000000,11.5,5.760000,267.800000,6.870000,...,4.220000,0.600000,0.400000,0.200000,31.026000,1.400000,10.800000,7.200000,69.300000,MIN
0,21.300000,11.500000,7.500000,2.300000,61.200000,359.300000,10.5,5.920000,236.100000,7.610000,...,4.260000,1.400000,0.800000,0.200000,30.287000,1.900000,10.500000,5.800000,64.000000,SF


In [26]:
stat_agg_df.to_csv('agg_team_stats.csv')